In [2]:
import requests
import googlemaps
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely as shp
from pyproj import Geod
from ast import literal_eval
import keyring

In [16]:
API_KEY = keyring.get_password("goomaps", "yianzhang14")

In [4]:
gmaps = googlemaps.Client(key=API_KEY)

In [5]:
# somehwere in manhattan
loc = (40.73188892751728, -73.98927188478527)

In [6]:
# using the healthcare facility general information from the ny data website
ny_hospitals = pd.read_csv("data/Health_Facility_General_Information.csv")
ny_hospitals.columns

Index(['Facility ID', 'Facility Name', 'Short Description', 'Description',
       'Facility Open Date', 'Facility Address 1', 'Facility Address 2',
       'Facility City', 'Facility State', 'Facility Zip Code',
       'Facility Phone Number', 'Facility Fax Number', 'Facility Website',
       'Facility County Code', 'Facility County', 'Regional Office ID',
       'Regional Office', 'Main Site Name', 'Main Site Facility ID',
       'Operating Certificate Number', 'Operator Name', 'Operator Address 1',
       'Operator Address 2', 'Operator City', 'Operator State',
       'Operator Zip Code', 'Cooperator Name', 'Cooperator Address',
       'Cooperator Address 2', 'Cooperator City', 'Cooperator State',
       'Cooperator Zip Code', 'Ownership Type', 'Facility Latitude',
       'Facility Longitude', 'Facility Location'],
      dtype='object')

In [7]:
# convert a latlon to a shapely point
def convert_to_point(row):
    if np.isnan(row["Facility Latitude"]):
        return shp.Point((0, 0))
    else:
        return shp.Point((float(row["Facility Longitude"]), float(row["Facility Latitude"])))

In [8]:
ny_hospitals["geometry"] = ny_hospitals.apply(lambda x: convert_to_point(x), axis=1)

In [9]:
ny_hospitals_gdf = gpd.GeoDataFrame(ny_hospitals, geometry="geometry", crs="EPSG:4326")

In [10]:
# filter out only nyc hospoitals
geod = Geod(ellps="WGS84")

In [11]:
ny_hospitals_gdf["dist_from_nyc"] = ny_hospitals_gdf["geometry"].apply(lambda x: geod.geometry_length(shp.geometry.LineString([x, shp.Point((loc[1], loc[0]))])))
ny_hospitals_gdf["dist_from_nyc"].describe()

count    6.132000e+03
mean     6.712690e+05
std      2.035252e+06
min      3.503383e+02
25%      1.621973e+04
50%      8.244176e+04
75%      3.344734e+05
max      8.666418e+06
Name: dist_from_nyc, dtype: float64

In [12]:
# extract hospitals less than 32 km away from point in manhattan
nyc_hospitals_gdf = ny_hospitals_gdf[ny_hospitals_gdf["dist_from_nyc"] <= 32000]
nyc_hospitals_gdf

,Facility ID,Facility Name,Short Description,Description,Facility Open Date,Facility Address 1,Facility Address 2,Facility City,Facility State,Facility Zip Code,...,Cooperator Address 2,Cooperator City,Cooperator State,Cooperator Zip Code,Ownership Type,Facility Latitude,Facility Longitude,Facility Location,geometry,dist_from_nyc
2,1370,The Riverside,NH,Residential Health Care Facility - SNF,09/28/1981,150 Riverside Drive,NaN,New York,New York,10024,...,NaN,NaN,New York,NaN,LLC,40.790638,-73.979767,"(40.790638, -73.979767)",POINT (-73.97977 40.79064),6573.223217
7,6035,Barnabas Hospital Methadone,HOSP-EC,Hospital Extension Clinic,07/18/1996,4535-39 Third Avenue,NaN,Bronx,New York,10457,...,NaN,NaN,New York,NaN,Not for Profit Corporation,40.854229,-73.890892,"(40.854229, -73.890892)",POINT (-73.89089 40.85423),15922.181955
15,7769,Sheepshead Bay Surgery Center,DTC,Diagnostic and Treatment Center,06/17/1999,2269 Ocean Avenue,NaN,Brooklyn,New York,11229,...,NaN,NaN,New York,NaN,Business Corporation,40.606426,-73.952667,"(40.606426, -73.952667)",POINT (-73.95267 40.60643),14271.999184
17,9013,Mobile Medical Unit,NaN,Mobile Hospital Extension Clinic,02/20/2009,460 Brielle Avenue,NaN,Staten Island,New York,10314,...,NaN,NaN,New York,NaN,Municipality,40.593884,-74.135338,"(40.593884, -74.135338)",POINT (-74.13534 40.59388),19683.052686
23,9970,"Parkmed NYC, LLC",DTC,Diagnostic and Treatment Center,10/31/2016,"800 Second Avenue, 6th Floor",NaN,New York,New York,10017,...,NaN,NaN,New York,NaN,LLC,40.750033,-73.971612,"(40.750033, -73.971612)",POINT (-73.97161 40.75003),2506.923922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6110,1369,New York Congregational Nursing Center,NH,Residential Health Care Facility - SNF,01/01/1901,135 Linden Boulevard,NaN,Brooklyn,New York,11226,...,NaN,NaN,New York,NaN,LLC,40.652481,-73.953758,"(40.652481, -73.953758)",POINT (-73.95376 40.65248),9315.021923
6118,13109,"Hand In Hand Homecare, Llc",LHCSA,Licensed Home Care Services Agency,06/23/2015,4200 White Plains Road,Suite 101,Bronx,New York,10466,...,NaN,NaN,New York,NaN,LLC,40.893549,-73.856988,"(40.893549, -73.856988)",POINT (-73.85699 40.89355),21139.065049
6119,6844,Planned Parenthood Project Street Beat Mobile ...,NaN,Mobile Diagnostic and Treatment Center Extensi...,01/13/2006,26 Bleecker Street,NaN,New York,New York,10012,...,NaN,NaN,New York,NaN,Not for Profit Corporation,40.725521,-73.993538,"(40.725521, -73.993538)",POINT (-73.99354 40.72552),793.692451
6121,12832,Home Instead,LHCSA,Licensed Home Care Services Agency,07/28/2005,400 E 56th St,Professional Wing Ste 2,New York,New York,10022,...,NaN,NaN,New York,NaN,Business Corporation,40.757538,-73.963374,"(40.757538, -73.963374)",POINT (-73.96337 40.75754),3591.260274


In [13]:
def get_place_details(row):
    place_id = gmaps.places(query=row["Facility Name"], location=literal_eval(row["Facility Location"]))["results"][0]["place_id"]
    return gmaps.place(place_id)

In [14]:
get_place_details(nyc_hospitals_gdf.iloc[0])

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '150',
    'short_name': '150',
    'types': ['street_number']},
   {'long_name': 'Riverside Drive',
    'short_name': 'Riverside Dr',
    'types': ['route']},
   {'long_name': 'Manhattan',
    'short_name': 'Manhattan',
    'types': ['sublocality_level_1', 'sublocality', 'political']},
   {'long_name': 'New York',
    'short_name': 'New York',
    'types': ['locality', 'political']},
   {'long_name': 'New York County',
    'short_name': 'New York County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'New York',
    'short_name': 'NY',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '10024', 'short_name': '10024', 'types': ['postal_code']}],
  'adr_address': '<span class="street-address">150 Riverside Dr</span>, <span class="locality">New York</span>, <spa